In [ ]:
# 1. 필수 라이브러리 설치 (gensim, konlpy 등 포함)
!pip install pandas konlpy gensim numpy

# 2. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 9.4 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import re
from tqdm import tqdm

# =============================================================================
# 1. 6대 지표 매핑 사전 (DNA 지표 계산)
# =============================================================================

INDICATOR_MAP = {
    # [결핍]
    'count_fatigue': [
        '지치다', '피곤하다', '힘들다', '어렵다', '난해하다', '복잡하다', '지루하다', '졸리다', '지친', '지쳐', '피곤',
        '읽기 힘든', '읽기 힘들', '이해하기 힘든', '이해 불가', '진도가 안', '머리 아픈', '머리 아파', '기빨림', '기빨려',
        '수면제', '잠온다', '집중 안', '포기', '하차', '덮었',
        '손이 안', '끝까지 못', '지겨워', '따분', '지쳐서', '장벽', '두꺼운', '쉽지 않'
    ],
    'count_disappointment': [
        '재미없다', '실망하다', '아깝다', '별로다', '최악이다', '노잼', '재미없', '실망', '실망스', '거품',
        '낚였', '낚임', '바이럴', '광고', '돈 아깝', '돈아깝', '돈낭비', '시간 아깝', '시간낭비', '시간아깝',
        '비추', '별로', '내용 없', '내용없', '빈약', '겉멋', '오타', '번역', '파손','실망을','의아','시시하','별거 없',
        '불만', '허위', '과장', '기대 이하', '기대만큼', '허접', '퀄리티', '실망감', '불만족', '환불', '괜히', '후회된'
    ],
    'count_skepticism': [
        '뻔하다', '진부하다', '당연하다', '추상적이다', '뜬구름', '당연한', '뻔한', '추상적', '비현실적', '와닿지',
        '괴리감', '적용하기 힘든', '적용 불가', '이상적', '누구나 아는', '다 아는', '도움 안', '그저 그런', '알맹이',
        '짜깁기', '원론적', '교과서','와닿지 않','식상한','진부하다','진부한',
        '식상', '뻔한 이야기', '새로운 내용이', '평이한', '뜬구름 잡', '내용 빈약', '실속이 없', '공허', '실천하기 힘든', '일반적인'
    ],
    # [충족]
    'count_healing': [
        '위로하다', '치유하다', '공감하다', '편안하다', '따뜻하다', '행복하다', '위로', '힐링', '따뜻한', '따스한',
        '감동', '눈물', '울컥', '치유', '공감', '먹먹', '마음', '행복', '선물', '편안', '다독', '안식', '휴식',
        '벅차', '위안', '가슴', '벅찬', '따뜻함', '뭉클', '힐링되는', '마음이', '든든', '위로받', '힘이', '눈물 펑펑'
    ],
    'count_insight': [
        '유익하다', '명확하다', '신선하다', '깊다', '통찰하다', '탁월하다', '유익', '명확', '신선', '깊이', '통찰',
        '지식', '배움', '알찬', '탁월', '논리적', '설득력', '깔끔', '완벽', '도움', '성장', '동기부여', '새로운 시각', '관점',
        '실용적', '실용성', '가이드', '인생의', '깨닫', '가르침', '방향', '정리된', '깔끔하게', '새롭게', '사고', '생각의'
    ],
    'count_fun': [
        '재미있다', '흥미롭다', '몰입하다', '빠져들다', '재밌', '재미있', '흥미', '몰입', '술술', '읽히다', '읽힌다',
        '순삭', '시간가는줄', '밤새', '가독성', '흡입력', '인생책', '강추', '추천', '소장', '필독',
        '완독', '앉은 자리에서', '단숨에', '흥미진진', '손에서 놓을 수', '최애', '다음편', '기다려', '페이지가', '호기심', '긴장감'
    ]
}

SAFE_PATTERNS = [
    '만나기 힘들', '찾기 힘들', '보기 힘들', '구하기 힘들',
    '힘들 때', '힘들 수', '힘들 정도로', '않아 힘들',
    '읽기 힘들지 않', '이해하기 힘들지 않', '지루하지 않', '재미없지 않'
]


# =============================================================================
# 헬퍼 함수: 컬럼 이름 유연하게 찾기
# =============================================================================
def sanitize_column_name(name):
    """컬럼 이름의 대소문자, 공백, 언더바, 점을 제거하고 소문자로 변환합니다."""
    if isinstance(name, str):
        return re.sub(r'[\s_\.]', '', name).lower()
    return name

def get_col(df, target_name):
    """정규화된 이름으로 컬럼을 찾아서 원래 이름을 반환합니다."""
    target_sanitized = sanitize_column_name(target_name)
    sanitized_cols = {sanitize_column_name(col): col for col in df.columns}

    if target_sanitized in sanitized_cols:
        return sanitized_cols[target_sanitized]
    return None

# =============================================================================
# 2. 분석 함수 (리뷰 단위 계산)
# =============================================================================

def count_indicators(row):
    """리뷰 텍스트를 기반으로 6대 감정 DNA 키워드 빈도를 계산합니다."""
    text_source = str(row.get('review_text', ''))
    is_safe = any(safe in text_source for safe in SAFE_PATTERNS)

    counts = {}
    for col_name, keywords in INDICATOR_MAP.items():
        count = 0
        is_negative_indicator = col_name in ['count_fatigue', 'count_disappointment', 'count_skepticism']

        # 안전 패턴이 발견되면 부정 지표 카운트를 0으로 처리 (오분류 방지)
        if is_negative_indicator and is_safe:
            count = 0
        else:
            for kw in keywords:
                count += text_source.count(kw)
        counts[col_name] = count
    return pd.Series(counts)

# =============================================================================
# 3. 폴더 처리 및 책 단위 요약 (Book Aggregator)
# =============================================================================

# 리뷰 데이터가 저장된 Google Drive 경로
input_folders = [
    '/content/drive/MyDrive/project/리뷰DB_KoELECTRA/'
]

def read_csv_with_encoding(file_path):
    """인코딩 오류를 방지하기 위해 utf-8, cp949 순으로 시도하는 헬퍼 함수"""
    try:
        return pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            return pd.read_csv(file_path, encoding='cp949')
        except Exception as e:
            raise Exception(f"Failed to read CSV with both utf-8 and cp949: {e}")

def aggregate_books(folder_list):
    print("🚀 [Start] 책 단위 데이터 통합 집계 (koELECTRA 기반) 시작...\n")

    POSITIVE_THRESHOLD = 0.5
    NEGATIVE_THRESHOLD = -0.5
    MAX_RATING_FOR_NORMALIZATION = 5.0

    for input_path in folder_list:
        folder_name = os.path.basename(input_path.rstrip(os.sep))
        clean_name = folder_name.replace('(token)', '')

        output_filename = f"{clean_name}_Book_Summary_koELECTRA.csv"
        output_path = os.path.join(os.path.dirname(input_path), output_filename)

        print(f"Target Folder: {folder_name}")
        print(f"Output File: {output_filename}")

        files = glob.glob(os.path.join(input_path, '*.csv'))
        if not files:
            print("파일이 없습니다.")
            continue

        book_summary_list = []

        for file_path in tqdm(files, desc="   └─ Aggregating"):
            file_base_name = os.path.basename(file_path)
            try:
                df = read_csv_with_encoding(file_path)

                # 컬럼 이름의 앞뒤 공백 제거
                df.columns = df.columns.str.strip()

                # 필수 KoELECTRA 컬럼 유연하게 찾기
                sentiment_col_name = get_col(df, 'sent_score_pm1')
                rating_col_name = get_col(df, 'rating_5pt')
                review_text_col_name = get_col(df, 'review_text')

                #  파일이 스킵될 때 컬럼 정보를 출력
                missing_cols_list = []
                if not sentiment_col_name: missing_cols_list.append('sent_score_pm1')
                if not rating_col_name: missing_cols_list.append('rating_5pt')
                if not review_text_col_name: missing_cols_list.append('review_text')

                if missing_cols_list:
                    print(f"\n    SKIPPED: {file_base_name}")
                    print(f"     └─ 누락 컬럼: {', '.join(missing_cols_list)}")
                    print(f"     └─ 파일 컬럼 목록: {list(df.columns)}") # 실제 파일 컬럼명을 확인합니다.
                    continue
                # 진단용 출력 추가 끝

                # 1. KoELECTRA 점수를 사용하여 sentiment_score 정의 (긍/부정 지표 대체)
                df['sentiment_score'] = df[sentiment_col_name].astype(float)

                # 2. 평점 컬럼 정의
                df['rating_val'] = df[rating_col_name].astype(float).fillna(0)

                # 거품 지수 계산
                df['bubble_index'] = (df['rating_val'] / MAX_RATING_FOR_NORMALIZATION) - df['sentiment_score']


                # 3. DNA 지표 계산을 위해 review_text 컬럼 정의
                df['review_text'] = df[review_text_col_name].fillna('')

                freq_df = df.apply(count_indicators, axis=1)
                df = pd.concat([df, freq_df], axis=1)

                # 4. 긍정/부정 리뷰 비율 계산 (KoELECTRA 점수 기반)
                total_reviews = len(df)
                if total_reviews == 0: continue

                pos_reviews = (df['sentiment_score'] >= POSITIVE_THRESHOLD).sum()
                neg_reviews = (df['sentiment_score'] <= NEGATIVE_THRESHOLD).sum()

                pos_ratio = round(pos_reviews / total_reviews, 3) if total_reviews > 0 else 0
                neg_ratio = round(neg_reviews / total_reviews, 3) if total_reviews > 0 else 0

                # 5. 책 정보 요약 - 메타데이터 추출 (유연하게 컬럼 찾기)
                title_col = get_col(df, 'title')
                author_col = get_col(df, 'author')
                category_col = get_col(df, 'category')
                publisher_col = get_col(df, 'publisher')

                book_meta = {
                    'Title': df[title_col].iloc[0] if title_col and not df[title_col].empty else file_base_name.replace('.csv', ''),
                    'Author': df[author_col].iloc[0] if author_col and not df[author_col].empty else 'Unknown',
                    'Category': df[category_col].iloc[0] if category_col and not df[category_col].empty else 'Unknown',
                    'Publisher': df[publisher_col].iloc[0] if publisher_col and not df[publisher_col].empty else 'Unknown',
                    'Review_Count': total_reviews
                }

                stats = {
                    'Sum_Fatigue': df['count_fatigue'].sum(),
                    'Sum_Disappointment': df['count_disappointment'].sum(),
                    'Sum_Skepticism': df['count_skepticism'].sum(),
                    'Sum_Healing': df['count_healing'].sum(),
                    'Sum_Insight': df['count_insight'].sum(),
                    'Sum_Fun': df['count_fun'].sum(),

                    'Avg_Rating': round(df['rating_val'].mean(), 2),
                    'Avg_Sentiment': round(df['sentiment_score'].mean(), 3),
                    'Avg_Bubble_Index': round(df['bubble_index'].mean(), 3),

                    'pos_review_ratio': pos_ratio,
                    'neg_review_ratio': neg_ratio
                }

                book_summary_list.append({**book_meta, **stats})

            except Exception as e:
                # 일반적인 처리 오류 출력
                print(f"\n    Error processing {file_base_name}: {e}")
                pass

        # 6. 저장 및 한글 컬럼 변환
        if book_summary_list:
            summary_df = pd.DataFrame(book_summary_list)

            # 한글 컬럼명 매핑
            rename_map = {
                'Title': '도서명', 'Author': '저자', 'Category': '분야', 'Publisher': '출판사', 'Review_Count': '총_리뷰_수',
                'Sum_Fatigue': '심리적_피로_빈도', 'Sum_Disappointment': '콘텐츠_실망_빈도', 'Sum_Skepticism': '실효성_부족_빈도',
                'Sum_Healing': '정서적_위로_빈도', 'Sum_Insight': '지적_통찰_빈도', 'Sum_Fun': '몰입감_재미_빈도',
                'Avg_Rating': '평균_평점', 'Avg_Sentiment': '평균_감성_점수', 'Avg_Bubble_Index': '평균_거품_지수',
                'pos_review_ratio': '강한_긍정_리뷰_비율',
                'neg_review_ratio': '강한_부정_리뷰_비율'
            }
            summary_df = summary_df.rename(columns=rename_map)

            # 최종 컬럼 순서 지정
            final_cols = [
                '도서명', '저자', '분야', '출판사', '총_리뷰_수',
                '심리적_피로_빈도', '콘텐츠_실망_빈도', '실효성_부족_빈도',
                '정서적_위로_빈도', '지적_통찰_빈도', '몰입감_재미_빈도',
                '평균_평점', '평균_감성_점수', '평균_거품_지수',
                '강한_긍정_리뷰_비율', '강한_부정_리뷰_비율'
            ]
            summary_df = summary_df[[c for c in final_cols if c in summary_df.columns]]

            summary_df.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f"  저장 완료: {output_path}\n")
        else:
            print("  저장할 데이터가 없습니다.\n")

    print(" koELECTRA 기반 분석 및 집계 작업이 모두 완료되었습니다!")

if __name__ == "__main__":
    aggregate_books(input_folders)

🚀 [Start] 책 단위 데이터 통합 집계 (koELECTRA 기반) 시작...

Target Folder: 리뷰DB_KoELECTRA
Output File: 리뷰DB_KoELECTRA_Book_Summary_koELECTRA.csv
파일이 없습니다.
 koELECTRA 기반 분석 및 집계 작업이 모두 완료되었습니다!


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# -----------------------------------------------------------
# 0. 환경 설정
# -----------------------------------------------------------
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

# -----------------------------------------------------------
# 1. 사용자 정의 변수 및 파일 설정
# -----------------------------------------------------------
# 생성된 통합 요약 파일 경로를 정확히 지정
SUMMARY_FILE_PATH = "/content/drive/MyDrive/project/리뷰DB_KoELECTRA_Book_Summary_koELECTRA.csv"
MIN_REVIEW_COUNT = 10  # 분석에 포함시킬 최소 리뷰 수

# 최종 CSV 파일의 한국어 컬럼을 분석에 사용할 영어/단축명으로 매핑
CORE_COLS_MAP = {
    '도서명': 'title',
    '저자': 'author',
    '분야': 'category',
    '총_리뷰_수': 'total_reviews',
    '평균_거품_지수': 'bubble_index',
    '평균_평점': 'avg_rating',
    '평균_감성_점수': 'avg_sentiment',
    '콘텐츠_실망_빈도': 'disappoint_freq',
    '실효성_부족_빈도': 'skepticism_freq',
}

# -----------------------------------------------------------
# 2. 데이터 로드 및 전처리
# -----------------------------------------------------------

print(f"통합 요약 파일 로드 시작: {os.path.basename(SUMMARY_FILE_PATH)}")

def read_summary_file(filename):
    """인코딩 오류를 처리하며 CSV 파일을 읽어오는 헬퍼 함수"""
    try:
        # 일반적으로 사용되는 UTF-8 BOM 인코딩으로 시도
        return pd.read_csv(filename, encoding='utf-8-sig')
    except:
        # 다른 인코딩으로 재시도
        return pd.read_csv(filename, encoding='cp949')

df = read_summary_file(SUMMARY_FILE_PATH)

if df is None or df.empty:
    raise FileNotFoundError("파일을 로드하지 못했거나 데이터가 비어있습니다. 경로와 파일 내용을 확인해주세요.")

# 컬럼명 정리 및 표준화
df.columns = df.columns.str.strip()
df_total_summary = df.rename(columns=CORE_COLS_MAP, errors='ignore')

# 필수 컬럼 검증
required_cols = list(CORE_COLS_MAP.values())
if not all(col in df_total_summary.columns for col in required_cols):
    missing_cols = [col for col in required_cols if col not in df_total_summary.columns]
    print(f"필수 분석 컬럼 누락: {missing_cols}")
    # 누락된 컬럼은 0으로 채우거나, 분석에서 제외되는 컬럼만 무시
    for col in ['disappoint_freq', 'skepticism_freq']:
        if col not in df_total_summary.columns:
            df_total_summary[col] = 0

print(f"최종 통합된 분석 데이터프레임 크기: {len(df_total_summary)} 행")

# 분석용 데이터 필터링 (최소 리뷰 수 충족)
df_filtered_analysis = df_total_summary[
    df_total_summary['total_reviews'] >= MIN_REVIEW_COUNT
].copy()
print(f"최소 리뷰 수({MIN_REVIEW_COUNT}개) 필터링 후: {len(df_filtered_analysis)} 행")


# -----------------------------------------------------------
# 3. 거품 도서 Top 10 선정 (bubble_top10_overall 변수 정의)
# -----------------------------------------------------------
bubble_top10_overall = df_filtered_analysis.sort_values(
    by='bubble_index',
    ascending=False,
    na_position='last'
).head(10)

print("\n=======================================================")
print("통합 베스트셀러 거품 도서 Top 10")
print("=======================================================")

# 평점-감성 점수 차이 계산
bubble_top10_overall['rating_sentiment_gap'] = (
    bubble_top10_overall['avg_rating'] - bubble_top10_overall['avg_sentiment']
)

final_result_top10 = bubble_top10_overall[[
    'title',
    'category', # 카테고리 추가
    'bubble_index',
    'avg_rating',
    'avg_sentiment',
    'rating_sentiment_gap',
    'total_reviews',
    'disappoint_freq',
    'skepticism_freq' # 실효성 부족 빈도 추가
]].copy()

print(final_result_top10.to_markdown(index=False, floatfmt=".3f"))


# -----------------------------------------------------------
# 4. 시각화 1: 거품 구간 Scatter Plot
# -----------------------------------------------------------

plt.figure(figsize=(14, 8))

# 리뷰 수에 따라 점 크기 설정
df_filtered_analysis['plot_size'] = df_filtered_analysis['total_reviews'] / 50
df_filtered_analysis['plot_size'] = df_filtered_analysis['plot_size'].clip(lower=10, upper=300)

plt.scatter(
    df_filtered_analysis['bubble_index'],
    df_filtered_analysis['avg_sentiment'],
    alpha=0.4,
    s=df_filtered_analysis['plot_size'],
    color='gray',
    label=f'전체 도서 (리뷰 {MIN_REVIEW_COUNT}개 이상)'
)

# Top 10 도서 시각화용 크기 조정
bubble_top10_overall['plot_size_top'] = bubble_top10_overall['total_reviews'] / 30
bubble_top10_overall['plot_size_top'] = bubble_top10_overall['plot_size_top'].clip(lower=50, upper=400)

plt.scatter(
    bubble_top10_overall['bubble_index'],
    bubble_top10_overall['avg_sentiment'],
    alpha=1.0,
    s=bubble_top10_overall['plot_size_top'],
    color='red',
    label='거품 도서 Top 10'
)

plt.title('통합 베스트셀러 거품 지수 vs. 독자 만족도', fontsize=18, pad=20)
plt.xlabel('거품 지수 (Bubble Index)', fontsize=14)
plt.ylabel('평균 감성 점수 (독자 만족도)', fontsize=14)
plt.axvline(x=df_filtered_analysis['bubble_index'].mean(), color='blue', linestyle=':', label='평균 거품 지수')
plt.axhline(y=df_filtered_analysis['avg_sentiment'].mean(), color='green', linestyle=':', label='평균 감성 점수')
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()


# -----------------------------------------------------------
# 5. 시각화 2: 카테고리별 거품 지수 집중도 분석
# -----------------------------------------------------------

category_bubble = df_filtered_analysis.groupby('category')['bubble_index'].mean().reset_index()
category_bubble = category_bubble.sort_values(by='bubble_index', ascending=False)
top_n_category = 10
category_bubble_top = category_bubble.head(top_n_category)

plt.figure(figsize=(12, 7))
sns.barplot(x='bubble_index', y='category', data=category_bubble_top, palette='viridis')
plt.title(f' 상위 {top_n_category}개 카테고리별 평균 거품 지수', fontsize=16)
plt.xlabel('평균 거품 지수', fontsize=12)
plt.ylabel('카테고리', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

print("\n=======================================================")
print("종합 분석 요약 결과 - Top 10 카테고리")
print("=======================================================")
print(category_bubble_top.to_markdown(index=False, floatfmt=".3f"))


# -----------------------------------------------------------
# 6. 심화 분석 1: Top 10 거품 도서 카테고리 분포 (파이 차트)
# -----------------------------------------------------------
print("\n=======================================================")
print("Top 10 거품 도서 카테고리 분포 분석")
print("=======================================================")

top10_category_counts = bubble_top10_overall['category'].value_counts().reset_index()
top10_category_counts.columns = ['Category', 'Count']
print(top10_category_counts.to_markdown(index=False))

plt.figure(figsize=(9, 9))
plt.pie(
    top10_category_counts['Count'],
    labels=top10_category_counts['Category'],
    autopct='%1.1f%%',
    startangle=140,
    textprops={'fontsize': 12},
    wedgeprops={'edgecolor': 'black', 'linewidth': 1}
)
plt.title('Top 10 거품 도서의 카테고리별 점유율', fontsize=18, pad=20)
plt.tight_layout()
plt.show()


# -----------------------------------------------------------
# 7. 심화 분석 2: 리뷰 수가 많은 도서 그룹의 거품 지수 통계 비교
# -----------------------------------------------------------
print("\n=======================================================")
print("리뷰 수 그룹별 거품 지수 통계 비교")
print("=======================================================")

# 리뷰 수 그룹화 (Quantile 기준)
df_filtered_analysis['review_quantile'] = pd.qcut(
    df_filtered_analysis['total_reviews'],
    q=[0, 0.5, 0.75, 1], # 50%, 50~75%, 75% 이상 그룹으로 분할
    labels=['하위 50%', '중위 25%', '상위 25%'],
    duplicates='drop'
)

bubble_by_review_group = df_filtered_analysis.groupby('review_quantile', observed=False)['bubble_index'].agg(['mean', 'median', 'count']).reset_index()
bubble_by_review_group.columns = ['리뷰_수_그룹', '평균_거품_지수', '중앙값_거품_지수', '도서_수']

print(bubble_by_review_group.to_markdown(index=False, floatfmt=".3f"))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------------------------------------------------------
# 1. 상관계수 계산 및 해석
# -----------------------------------------------------------------------------

if 'df_total_summary' in locals() and not df_total_summary.empty:

    # 1. 계산에 사용할 컬럼을 추출하고 NaN 값 제거
    correlation_df = df_total_summary[['popularity_index', 'avg_sentiment']].dropna()

    if correlation_df.empty:
        print("경고: 상관계수 계산에 사용할 유효한 데이터(popularity_index, avg_sentiment)가 없습니다.")
    else:
        # 2. 피어슨 상관계수 계산 (rho)
        correlation_coefficient = correlation_df['popularity_index'].corr(
            correlation_df['avg_sentiment']
        )

        # 3. 결과 해석
        if abs(correlation_coefficient) < 0.2:
            interpretation = "매우 약한 상관관계 또는 거의 무관함"
        elif 0.2 <= abs(correlation_coefficient) < 0.4:
            interpretation = "약한 상관관계"
        else:
            interpretation = "보통 이상의 상관관계"

        print("================================================================")
        print(" 베스트셀러 순위(인기 지수) vs. 독자 만족도(감성 점수) 상관 분석")
        print("================================================================")
        print(f"전체 도서 수 (계산에 사용): {len(correlation_df)}개")
        print(f"피어슨 상관계수 (ρ): {correlation_coefficient:.4f}")
        print(f"해석: {interpretation}")
        print(" 상관계수가 0에 가깝다는 것은, 책의 순위와 독자의 감성 만족도는 거의 무관하다는 것을 의미합니다.")